# Evaluation

The evaluation file use JupyterNotebook to visual the simulation and train result.

In [ ]:
from settings import APP_CONFIG, UNREAL_CONFIG

CHECKPOINT_PATH = APP_CONFIG['checkpoint_path']
HISTORY_PATH = APP_CONFIG['training_history']
CSV_PATH = APP_CONFIG['preprocessed_csv_file']
NUM_OF_BEACONS = UNREAL_CONFIG['num_of_beacons']

## 1. Loss

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

In [ ]:
def plot_loss(df):
    plt.figure(figsize=(8,8))
    plt.plot(df.training_loss)
    plt.plot(df.test_loss)
    plt.legend(labels=['training loss','test loss'], loc='upper left')

In [ ]:
loss_df = pd.read_csv(HISTORY_PATH)
plot_loss(loss_df)

Load csv data to dataframe

In [ ]:
import pandas as pd

df = pd.read_csv(CSV_PATH)
df.head()

## 2. Load Trained Model

In [ ]:
import tensorflow as tf
from nn.deepAR import create_model

model = create_model(dim_x=NUM_OF_BEACONS, dim_z=3)
ckpt = tf.train.Checkpoint(model=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_PATH, max_to_keep=100)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Restoring checkpoint from {}".format(ckpt_manager.latest_checkpoint))

## 3. Visualize Prediction Result

### 3.1 Contruct Long Sequence Dataset

In [ ]:
from preprocessing.dataset import DatasetGenerator

norm_df = df.drop(columns=['timestamp'])
mean, std = list(norm_df.mean()[:3]), list(norm_df.std()[:3])
norm_df = (norm_df - norm_df.mean())/(norm_df.std() + 1e-10)
columns = list(norm_df.columns)
seq_len = 2000
long_dg = DatasetGenerator(data=norm_df, input_width=seq_len, label_width=seq_len, 
                           shift=0, batch_size=1, columns=columns, input_columns=columns[3:], label_columns=columns[:3],  partition=(0.8,0.1,0.1))

In [ ]:
import numpy as np

def model_plot2D(model, data, batch_index, title='',warmup_step=10, df_mean=0, df_std=1, x_offset=0, y_offset=0):
    plt.figure(figsize=(20,10))
    inputs, labels = data
    means, _ = model(inputs)
    mean = (means[batch_index, :] * df_std + df_mean) * 0.01
    label = (labels[batch_index, :] * df_std + df_mean) * 0.01
    plt.plot(-mean[warmup_step:, 0] + x_offset, mean[warmup_step:, 1] + y_offset, '-r', alpha=0.5)
    plt.plot(-label[warmup_step:,0] + x_offset, label[warmup_step:,1] + y_offset, '--g', alpha=0.5)
    plt.legend(labels=['Prediction', 'Ground Truth'], loc='lower left')
    plt.xlabel('X Coordinate (m)')
    plt.ylabel('Y Coordinate (m)')
    plt.title(title)
    plt.axis('equal')
    plt.show()

### 3.2 Training-set Result

In [ ]:
assert len(list(long_dg.train_ds)) > 0, "Data size is too small to contruct training-set, please try to collect more data and try again"
model_plot2D(model=model, data=next(iter(long_dg.train_ds)), title='Trajectory Forecasting with Training-set',batch_index=0, df_mean=mean, df_std = std, x_offset=11.5, y_offset=6)

### 3.3 Test-set Result

In [ ]:
assert len(list(long_dg.test_ds)) > 0, "Data size is too small to contruct test-set, please try to collect more data and try again"
model_plot2D(model=model, data=next(iter(long_dg.test_ds)), title='Trajectory Forecasting with Test-set', batch_index=0, df_mean=mean, df_std = std, x_offset=11.5, y_offset=6)